In [ ]:
import logging
logging.basicConfig(level=logging.DEBUG)

In [ ]:
"""Put config.json with ml workspace creds to the root"""
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

ml_client = MLClient.from_config(credential=DefaultAzureCredential(), path="../")

In [ ]:
"""Create one place with inputs to the pipelines"""
from azure.ai.ml import Input
from azure.ai.ml.constants import InputOutputModes as IOMode

input_data = Input(path="azureml://datastores/raw/paths/data", mode=IOMode.DIRECT)
markers = "AAPL"
output_data = Input(path="azureml://datastores/cleaned/paths/data", mode=IOMode.DIRECT)
subscription_id = ml_client.subscription_id
resource_group = ml_client.resource_group_name
workspace_name = ml_client.workspace_name
client_id = "" ### Set client_id of user assigned identity
keyvault_name = "stockpredict-ml" ### Set keyvault name
train_input = Input(path="azureml://datastores/cleaned/paths/data", mode=IOMode.RO_MOUNT)
epochs = 10

In [ ]:
"""Helper"""
def start_pipeline(pipe) -> None:
    pipe.compute = "serverless"
    job = ml_client.jobs.create_or_update(pipeline)
    ml_client.jobs.stream(job.name)

In [ ]:
"""Fetch pipeline"""
from stockpredict.deployment.pipelines import fetch_data_pipeline
pipeline = fetch_data_pipeline(input_path=input_data, markers=markers ,keyvault_name=keyvault_name)
start_pipeline(pipeline)

In [ ]:
"""ETL pipeline"""
from stockpredict.deployment.pipelines import etl_pipeline
pipeline = etl_pipeline(input_data=input_data, output_data=output_data)
start_pipeline(pipeline)

In [ ]:
"""Train / Register model pipeline"""
from stockpredict.deployment.pipelines import train_register_pipeline
pipeline = train_register_pipeline(input_data=train_input, subscription_id=subscription_id, resource_group=resource_group, workspace_name=workspace_name, client_id=client_id, epochs=epochs)
start_pipeline(pipeline)

In [ ]:
"""Fetch -> ETL -> Train -> Register model -> Deploy pipeline"""
from stockpredict.deployment.pipelines import fetch_etl_train_register_deploy_pipeline
pipeline = fetch_etl_train_register_deploy_pipeline(input_data=input_data, subscription_id=subscription_id, resource_group=resource_group, workspace_name=workspace_name, client_id=client_id, markers=markers, output_data=output_data, keyvault_name=keyvault_name, epochs=epochs)
start_pipeline(pipeline)